In [1]:
# === Task 3, Cell 1: Imports (Final - Using FQEConfig) ===
import pandas as pd
import numpy as np
import joblib
import torch

from sklearn.model_selection import train_test_split

# The Offline Reinforcement Learning library
import d3rlpy

# d3rlpy components for the algorithm
from d3rlpy.dataset import MDPDataset
from d3rlpy.algos import DiscreteCQLConfig

# --- CORRECTED ---
# Components for DISCRETE Off-Policy Evaluation (OPE)
# We use the Discrete evaluator but the generic FQEConfig
from d3rlpy.ope import FQEConfig # The generic config class
from d3rlpy.ope import DiscreteFQE
from d3rlpy.metrics import InitialStateValueEstimationEvaluator
from d3rlpy.preprocessing import MinMaxRewardScaler

use_gpu = torch.cuda.is_available()
print(f"Using GPU: {use_gpu}  (expected: False for Intel Iris)")
print(f"d3rlpy version: {d3rlpy.__version__}")
print("All libraries imported successfully.")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
C:\Users\Asus\miniconda3\envs\shodh_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using GPU: False  (expected: False for Intel Iris)
d3rlpy version: 2.8.1
All libraries imported successfully.


In [2]:
# === Task 3, Cell 2: Load Data (Corrected Filename & Rewards) ===

# 1. Load the *original* (unscaled) data again
data_path = '../data/raw/accepted_2007_to_2018Q4.csv' # Corrected filename

try:
    # Add low_memory=False to suppress the DtypeWarning
    df = pd.read_csv(data_path, nrows=500000, low_memory=False)
except FileNotFoundError:
    print(f"ERROR: File not found at {data_path}")
    print("Please make sure your file is named correctly and in the 'data/raw' folder.")
    raise

print(f"Original raw data loaded: {df.shape}")

# 2. Re-create the binary 'target' variable (same as Task 1)
bad_statuses = [
    'Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off'
]
good_statuses = [
    'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'
]

df['target'] = np.nan
df.loc[df['loan_status'].isin(good_statuses), 'target'] = 0
df.loc[df['loan_status'].isin(bad_statuses), 'target'] = 1
df_clean = df.dropna(subset=['target'])
df_clean = df_clean.astype({'target': int})

print(f"Data filtered to good/bad loans: {df_clean.shape}")

# 3. Define the RL components
FEATURES_TO_KEEP = [
    'loan_amnt', 'term', 'int_rate', 'annual_inc', 'dti', 'emp_length',
    'pub_rec', 'revol_util', 'total_acc', 'open_acc', 'mort_acc', 'fico_range_low'
]
TARGET_COL = 'target'

df_rl = df_clean[FEATURES_TO_KEEP + [TARGET_COL]].copy()


# --- 4. Engineer State, Action, Reward ---
# Clean categorical and fill missing
df_rl['term'] = df_rl['term'].str.replace(' months', '', regex=False).astype(int)
df_rl['emp_length'] = df_rl['emp_length'].str.replace('< 1 year', '0 years', regex=False)
df_rl['emp_length'] = df_rl['emp_length'].str.replace('10+ years', '10 years', regex=False)
df_rl['emp_length'] = df_rl['emp_length'].str.extract(r'(\d+)').astype(float)
df_rl = df_rl.fillna(df_rl.median())

# Load the scaler
scaler_path = '../models/scaler.joblib'
scaler = joblib.load(scaler_path)

# Create scaled observations (state)
observations = scaler.transform(df_rl[FEATURES_TO_KEEP])
print(f"States (observations) created and scaled: {observations.shape}")

# Create actions (always 1 for historical data)
actions = np.ones(len(df_rl), dtype=int)
print(f"Actions created (all 1s): {actions.shape}")

# Create rewards (raw dollar value)
profit = df_rl['loan_amnt'] * (df_rl['int_rate'] / 100.0)
loss = -df_rl['loan_amnt']
rewards = np.where(
    df_rl['target'] == 0, # Condition: Was the loan 'Fully Paid'?
    profit,               # If true: reward = profit
    loss                  # If false: reward = loss
)
print(f"Rewards engineered (raw $ value). Average reward: ${rewards.mean():.2f}")

# Create terminals (always 1 for this single-step problem)
terminals = np.ones(len(df_rl), dtype=int)
print("All RL arrays created.")

Original raw data loaded: (500000, 151)
Data filtered to good/bad loans: (391168, 152)


C:\Users\Asus\miniconda3\envs\shodh_env\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


States (observations) created and scaled: (391168, 12)
Actions created (all 1s): (391168,)
Rewards engineered (raw $ value). Average reward: $-1806.30
All RL arrays created.


In [3]:
# === Task 3, Cell 3: Split Arrays and Create Datasets ===

# 1. Split all four arrays at the same time
#    We use the scikit-learn train_test_split we imported in Cell 1
obs_train, obs_test, act_train, act_test, rew_train, rew_test, term_train, term_test = train_test_split(
    observations, 
    actions, 
    rewards,  # Use the raw (unscaled) rewards
    terminals, 
    test_size=0.2, 
    random_state=42
)

print("--- RL Arrays Split ---")

# 2. Now, create two separate MDPDataset objects
train_dataset = MDPDataset(
    observations=obs_train,
    actions=act_train,
    rewards=rew_train,
    terminals=term_train
)

test_dataset = MDPDataset(
    observations=obs_test,
    actions=act_test,
    rewards=rew_test,
    terminals=term_test
)

print("--- MDPDataset Objects Created ---")
# --- FIXED ---
# We must use the .size() method, not len()
print(f"Training dataset size: {train_dataset.size()}")
print(f"Test dataset size: {test_dataset.size()}")

--- RL Arrays Split ---
2025-10-30 14:37.42 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(12,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-10-30 14:37.42 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-30 14:37.45 [info     ] Action size has been automatically determined. action_size=2
2025-10-30 14:37.46 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(12,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-10-30 14:37.46 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-30 14:37.47 [info     ] Action size has been automatically determined

In [4]:
# === Task 3, Cell 4: Define the RL Algorithm (Discrete CQL) ===
from d3rlpy.preprocessing import MinMaxRewardScaler # Make sure scaler is imported

# 1. Set up the configuration for the Discrete CQL algorithm.
#    Hyperparameters like learning rate and reward scaling are set here.
cql_config = DiscreteCQLConfig(
    reward_scaler=MinMaxRewardScaler(), # Use the imported class instance
    learning_rate=3e-4                  # Optimizer learning rate
)

# 2. Initialize the Discrete CQL algorithm object using the configuration.
#    Pass use_gpu to determine if it should run on CPU or GPU.
cql = cql_config.create(device=use_gpu)

print("--- Discrete CQL Algorithm Initialized ---")
print(f"Algorithm: {cql.__class__.__name__}")
print(f"Using GPU: {use_gpu}")

--- Discrete CQL Algorithm Initialized ---
Algorithm: DiscreteCQL
Using GPU: False


In [5]:
# === Task 3, Cell 5: Train the RL Agent ===

print("--- Starting RL Model Training ---")
print("This will take several minutes. A progress bar will appear.")

# Determine the number of training steps (equivalent to 1 epoch).
n_steps_for_one_epoch = train_dataset.size()

print(f"Training for {n_steps_for_one_epoch} steps (equivalent to 1 epoch)...")

# Train the CQL model using the 'n_steps' argument.
cql.fit(
    train_dataset,
    n_steps=n_steps_for_one_epoch
)

print("\n--- RL Model Training Complete ---")

--- Starting RL Model Training ---
This will take several minutes. A progress bar will appear.
Training for 312934 steps (equivalent to 1 epoch)...
2025-10-30 14:38.11 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(12,)]), action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-10-30 14:38.11 [debug    ] Fitting reward scaler...       reward_scaler=min_max
2025-10-30 14:38.17 [debug    ] Building models...            
2025-10-30 14:38.21 [debug    ] Models have been built.       
2025-10-30 14:38.21 [info     ] Directory is created at d3rlpy_logs\DiscreteCQL_20251030143821
2025-10-30 14:38.21 [info     ] Parameters                     params={'observation_shape': [12], 'action_size': 2, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 32, 'gamma': 0.99, 

Epoch 1/31: 100%|█| 10000/10000 [01:52<00:00, 89.28it/s, loss=0.0142, td_loss=0.0126, conse


2025-10-30 14:40.13 [info     ] DiscreteCQL_20251030143821: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.0019885479211807252, 'time_algorithm_update': 0.008737970757484436, 'loss': 0.014151464512199164, 'td_loss': 0.01256833084761165, 'conservative_loss': 0.0015831336646922865, 'time_step': 0.011059957265853881} step=10000
2025-10-30 14:40.13 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20251030143821\model_10000.d3


Epoch 2/31: 100%|█| 10000/10000 [02:05<00:00, 79.68it/s, loss=0.0113, td_loss=0.0113, conse


2025-10-30 14:42.19 [info     ] DiscreteCQL_20251030143821: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.002066092586517334, 'time_algorithm_update': 0.00999345896244049, 'loss': 0.01126854825859191, 'td_loss': 0.011267116616491693, 'conservative_loss': 1.4316409826278687e-06, 'time_step': 0.012415913915634156} step=20000
2025-10-30 14:42.19 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20251030143821\model_20000.d3


...

2025-10-30 15:58.10 [info     ] DiscreteCQL_20251030143821: epoch=29 step=290000 epoch=29 metrics={'time_sample_batch': 0.0018761574983596802, 'time_algorithm_update': 0.014860567116737366, 'loss': 0.010711822089995257, 'td_loss': 0.010711314086266794, 'conservative_loss': 5.080035247374326e-07, 'time_step': 0.017095691204071044} step=290000
2025-10-30 15:58.10 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20251030143821\model_290000.d3


Epoch 30/31: 100%|█| 10000/10000 [02:53<00:00, 57.66it/s, loss=0.0107, td_loss=0.0107, cons


2025-10-30 16:01.03 [info     ] DiscreteCQL_20251030143821: epoch=30 step=300000 epoch=30 metrics={'time_sample_batch': 0.0019646296977996825, 'time_algorithm_update': 0.01470176112651825, 'loss': 0.010723373251478189, 'td_loss': 0.010722285219444893, 'conservative_loss': 1.0880318703129888e-06, 'time_step': 0.017041891741752625} step=300000
2025-10-30 16:01.04 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20251030143821\model_300000.d3


Epoch 31/31: 100%|█| 10000/10000 [02:53<00:00, 57.76it/s, loss=0.0107, td_loss=0.0107, cons


2025-10-30 16:03.57 [info     ] DiscreteCQL_20251030143821: epoch=31 step=310000 epoch=31 metrics={'time_sample_batch': 0.0019162012100219727, 'time_algorithm_update': 0.014773324632644653, 'loss': 0.010719549461535643, 'td_loss': 0.010718521325092297, 'conservative_loss': 1.028136210516095e-06, 'time_step': 0.017038668990135192} step=310000
2025-10-30 16:03.57 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20251030143821\model_310000.d3

--- RL Model Training Complete ---


In [ ]:
# === Task 3, Cell 6: Evaluate (This is the correct, final version) ===

print("--- Starting Off-Policy Evaluation (DiscreteFQE) ---")

# 1. Create the generic FQEConfig object.
#    This assumes Cell 1 has imported FQEConfig.
fqe_config = FQEConfig(
    learning_rate=3e-4,
    reward_scaler=MinMaxRewardScaler()
)

# 2. Create DiscreteFQE, passing the trained algo and the config object.
fqe = DiscreteFQE(
    algo=cql,                  # Your trained CQL agent
    config=fqe_config,         # The config object
    device=use_gpu
)

# 3. Create the evaluator object to be passed in.
initial_state_evaluator = InitialStateValueEstimationEvaluator()

print("DiscreteFQE algorithm initialized. Training FQE model...")

# 4. Train the FQE model AND evaluate it at the same time.
#    The evaluator is passed in the 'evaluators' dictionary.
n_test_steps = test_dataset.size()
fqe.fit(
    test_dataset,
    n_steps=n_test_steps,
    evaluators={
        'policy_value': initial_state_evaluator # Pass the scorer HERE
    }
)

print("DiscreteFQE training complete.")

# 5. Get the final "Estimated Policy Value" from the log history.
#    The fit process saves the result in fqe.log_history
policy_value = fqe.log_history[-1]['policy_value']

print("\n--- RL Model Evaluation Complete ---")
# Get the raw historical average reward
avg_historical_reward_raw = test_dataset.rewards.mean()

# Unscale the policy value
min_reward = rewards.min() # 'rewards' is the unscaled array from Cell 2
max_reward = rewards.max()
policy_value_unscaled = policy_value * (max_reward - min_reward) + min_reward

print(f"   Estimated Policy Value (Unscaled $): ${policy_value_unscaled:.2f}")
print(f"Average Historical Reward (Raw $):    ${avg_historical_reward_raw:.2f}")

--- Starting Off-Policy Evaluation (DiscreteFQE) ---
DiscreteFQE algorithm initialized. Training FQE model...
2025-10-30 16:34.21 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(12,)]), action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-10-30 16:34.21 [debug    ] Fitting reward scaler...       reward_scaler=min_max
2025-10-30 16:34.22 [debug    ] Building models...            
2025-10-30 16:34.22 [debug    ] Models have been built.       
2025-10-30 16:34.22 [info     ] Directory is created at d3rlpy_logs\DiscreteFQE_20251030163422
2025-10-30 16:34.22 [info     ] Parameters                     params={'observation_shape': [12], 'action_size': 2, 'config': {'type': 'fqe', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params

Epoch 1/7: 100%|████████████████████████| 10000/10000 [03:27<00:00, 48.11it/s, loss=0.0111]


2025-10-30 16:40.27 [info     ] DiscreteFQE_20251030163422: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.005744779515266418, 'time_algorithm_update': 0.013789256811141968, 'loss': 0.011077759804530069, 'time_step': 0.020053118991851807, 'policy_value': 0.7486783862113953} step=10000
2025-10-30 16:40.27 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteFQE_20251030163422\model_10000.d3


Epoch 2/7: 100%|███████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:04<00:00, 155.64it/s, loss=0.0102]


2025-10-30 16:42.23 [info     ] DiscreteFQE_20251030163422: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.0026896746635437013, 'time_algorithm_update': 0.003498509955406189, 'loss': 0.010233220964111387, 'time_step': 0.006355924654006958, 'policy_value': 0.7469315528869629} step=20000
2025-10-30 16:42.23 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteFQE_20251030163422\model_20000.d3


Epoch 3/7: 100%|██████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:05<00:00, 153.21it/s, loss=0.00937]


2025-10-30 16:44.20 [info     ] DiscreteFQE_20251030163422: epoch=3 step=30000 epoch=3 metrics={'time_sample_batch': 0.002661634063720703, 'time_algorithm_update': 0.0036179882526397706, 'loss': 0.00937036258103326, 'time_step': 0.006448856854438782, 'policy_value': 0.7373618483543396} step=30000
2025-10-30 16:44.20 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteFQE_20251030163422\model_30000.d3


Epoch 4/7:   9%|████████▍                                                                                | 944/10000 [00:24<03:46, 39.99it/s, loss=0.00891]